# Setup

In [ ]:
from __future__ import print_function
from tqdm.notebook import tqdm
import pandas as pd
import glob
import time
import numpy as np
import os
import math
import pprint

# Load Brand Analytics Data

In [ ]:
fileNameBA = "./Amazon-Suchbegriffe_DE.csv" # DE
thousandSeparator = "." # DE
columns = ["Suchbegriff","Suchfrequenz-Rang ","#1 der angeklickten ASINs","#2 der angeklickten ASINs","#3 der angeklickten ASINs"] # DE
# columns = ["Search Term","Search Frequency Rank","#1 Clicked ASIN","#2 Clicked ASIN","#3 Clicked ASIN"] # US

# Load data
dfBA = pd.read_csv(fileNameBA, thousands=thousandSeparator, usecols=columns, engine="python", error_bad_lines=True, encoding='utf-8', skiprows=1,  sep=",")

# Rename columns
dfBA.columns = ['searchterm', 'rank', '1', '2', '3']

# Unmelt dfBA from wide to long
dfBA_Long = dfBA.melt(id_vars=["searchterm", "rank"], var_name="position", value_name="ASIN")

# Make position an int
dfBA_Long = dfBA_Long.astype({"position": int})

# Drop N/A
dfBA_Long = dfBA_Long.dropna()

# Reset index and sort
dfBA_Long_WithIndex = dfBA_Long.set_index('searchterm')
dfBA_Long_WithIndex = dfBA_Long_WithIndex.sort_index()

In [ ]:
print(dfBA_Long_WithIndex.sample(4).to_markdown())

# Load products data

In [ ]:
fileNameProducts = "./produkt-listings.csv"
columns = ["Marketplace", "ASIN (child)","Product Title", "Bullet Point 1","Bullet Point 2","Bullet Point 3","Bullet Point 4","Bullet Point 5"]
df_Products = pd.read_csv(fileNameProducts, usecols=columns, engine="python", error_bad_lines=True, encoding='utf-8', sep=",")

# Only use DE products
df_Products = df_Products[df_Products["Marketplace"] == 'DE']

# For loop

In [ ]:
# Loop through all products in df_Products

for ASIN in tqdm(df_Products["ASIN (child)"], desc="Looping through products..."):
    
    # Find keywords the ASIN is clicked for
    ownedKeywords = []
    foundKeywords = dfBA_Long[dfBA_Long['ASIN'] == ASIN].searchterm.unique()
    ownedKeywords.append(foundKeywords)

    # Get other ASINs from competitors for ownedKeywords
    otherASINs = []
    for searchterm in ownedKeywords[0]:
        # print(searchterm)
        # foundASINs = dfBA_Long[dfBA_Long['searchterm'] == searchterm].ASIN.unique().flatten()
        
        try:
            temp = dfBA_Long_WithIndex.loc[searchterm].ASIN
            if isinstance(temp, str):
                # Result is a string
                foundASINs = temp
            else:
                # Is an array
                foundASINs = temp.unique().flatten()
            otherASINs.append(foundASINs)
        except KeyError:
            pass  # do nothing!
        
    # Flatten array of arrays
    flat_list_ASINs = [item for sublist in otherASINs for item in sublist]

    # Make array unique using a set
    flat_list_ASINs = set(flat_list_ASINs)

    # Remove current (own) ASIN
    try:
        flat_list_ASINs.remove(ASIN)
    except KeyError:
        pass  # do nothing!

    # Get unique list of keywords the other ASINs are clicked on
    keywordsFromOtherASINs = dfBA_Long[dfBA_Long['ASIN'].isin(flat_list_ASINs)].searchterm.unique()

    # Get keywords which are other ASINs clicked on but not the own ASIN yet
    A = np.array(ownedKeywords)
    B = np.array(keywordsFromOtherASINs)
    missingKeywords = np.setdiff1d(B, A)

    # Flatten array of arrays
    missingKeywords_flattened = set(' '.join(missingKeywords).split(' '))

    # Only get keywords which have a minimum length of 4
    missingKeywords_flattened_reduced = [str for str in missingKeywords_flattened if len(str) >= 4]
    missingKeywords_flattened_reduced.sort()
    
    # Get the product title for ASIN in question
    productTitle = df_Products[df_Products['ASIN (child)'] == ASIN]['Product Title'].values[0].lower()

    # Get all bullet points as one string
    allBullets = []
    currentProduct = df_Products[df_Products["ASIN (child)"] == ASIN]
    for i in range (1,6):
        if not pd.isna(currentProduct['Bullet Point ' + str(i)].values[0]):
            allBullets.append(currentProduct['Bullet Point ' + str(i)].values[0])

    if len(allBullets) > 0:
        allBulletsCombined = ' '.join(allBullets)

    # Check if a term from missingKeywords_flattened_reduced is not in product title or bullet points
    termsNotFoundInListing = []
    for term in missingKeywords_flattened_reduced:
        if (term.lower() not in productTitle) and (term.lower() not in allBulletsCombined.lower()):
            termsNotFoundInListing.append(term)

    # Add missing keywords to df_Products dataframe
    df_Products.loc[df_Products["ASIN (child)"] == ASIN, 'missing'] = str(termsNotFoundInListing)

In [ ]:
# Show products with keyword recommendations
suggestions = df_Products[df_Products.missing.str.len() > 2]
suggestions

In [ ]:
# Write result to disk
timeString = time.strftime("%Y%m%d-%H%M%S")
fileName = timeString + '-Keyword-Suggestions.xlsx'

with pd.ExcelWriter(fileName) as writer:  
    suggestions.to_excel(writer, sheet_name='Suggestions')